# FISHERIES MODELLING

The main goal in this code is to create the methodology to model fish stock change with different climate change scenarios combining the Exploitable Biommas Index that can be obtained from the SPiCT model, envrionmental variables and a machine learning algorithm (potentially GAMs). As the SPiCT model results should be used in relative terms, the fish stock changes in Climate Change scenarios should also be reported in relative terms.

The steps that we have to follow are the following ones:
1. Prepare environmental data.
2. Predictor selection
3. Model training, testing and validation
4. Run projections

Python version: 3.1.2

All results will be saved in a new folder called 'Fisheries_modelling' saved in the working directory.

## ENVIRONMENTAL DATA PREPARATION

In [1]:
# Import the needed libraries to prepare the environmental data:
import os 
import xarray as xr
import geopandas as gpd
import pandas as pd
from shapely.geometry import mapping
import re
import numpy as np


In [2]:
# User input code-block:

    # Xarray is not able to identify the EPSG code or coordinate system of the Copernicus netCDF, so the user should manually check the EPSG code in a GIS software and introduce it:

        # Environmental variables EPSG code:
env_epsg = f"EPSG:{input('Environmental variables EPSG code:')}"

    # Working directory where environmental variable netCDFs are:
wd = r"C:\PhD\Copernicus_env_data_fisheries\Cadiz"

    # First prediction year (year after last landings data):
start_year = float(input('First prediction year:'))

    # First envionmental data time:
env_data_start = float(input('First environmental data year:')) # NOTICE that if the first data is not in 1rst January, as the input has to be floating type, you should use 'input = year + ((month_first_data - 1)/12)' to compute input.

    # Name of the shapefile that delimits the region of interest:
roi_file = f"{input('Name of the shapefile that delimits the region of interest:')}"
        # Ensure the shapefile name is correct:
if '.shp' in roi_file:
    pass
else:
    roi_file = roi_file + '.shp'

    # Name of the csv where the relative biomass estimates obtained from the SPiCT model are stored:
spict_data = f"{input("Name of the csv where the SPiCT estimates and date of estimates are:")}"
        # Ensure the csv name is correct:
if '.csv' in spict_data:
    pass
else: 
    spict_data = spict_data + '.csv'

    # Biomass at Maximum Sustainable Yield obtained from SPiCT:
bmsy = float(input('Biomass at Maximum Sustainable Yield computed from SPiCT model:'))

In [3]:
# Set the working directory:
os.chdir(wd)
# Define the file names and store the netCDFs in an array:
netcdfs = [nc for nc in os.listdir(wd) if ".nc" in nc[-3:]]

In [ ]:
# Read the csv file where we have the SPiCT relative biomass estimates as well as the date of each estimation:
dataset = pd.read_csv(spict_data)

# Clean the SPiCT dataset to keep just estimates of the dates where environmental data are available (also delete predictions of the SPiCT):
dataset = dataset.drop(dataset[(dataset['time'] > start_year) | (dataset['time'] < env_data_start)].index)

# Function to convert decimal years of the csv to date type data:
def decimal_year_to_ym(decimal_year):
    year = np.floor(decimal_year).astype(int) 
    remainder = decimal_year - year  
    month = np.floor(remainder * 12).astype(int) + 1 
    return pd.to_datetime(year.astype(str) + '-' + month.astype(str).str.zfill(2), format='%Y-%m')

# Execute function to change 'time' column to a date type:
dataset['time'] = decimal_year_to_ym(dataset['time'])

# If two estimates are for one month compute the average to have just 1 relative biomass estimate by month:
dataset_cleared = dataset.groupby('time', as_index=False)['relative_B'].mean()  # Notice that 'time' and 'relative_B' are the field names in the original SPiCT estimate csv

# Accumulator:
i_bio = 0
i_phy = 0

# Loop the dataset and make the needed computations to prepare the input dataset:

for env_netcdf in netcdfs:
    # 1. Open the dataset and define the coordinate system:
    try:
        ds = xr.open_dataset(env_netcdf, engine="h5netcdf")
        ds = ds.rio.write_crs(env_epsg, inplace=True)
    except Exception as e:
        print(f"Error opening file with xarray: {e}")

    # 2. Mask the dataset to keep just the depth ranges that will be used:
    depth_mask_up = ((ds['depth'] > 5) & (ds['depth'] < 10))
    depth_mask_low = ((ds['depth'] > 45) & (ds['depth'] < 100))

    # 3. Filter the datasets to keep just the desired depths:
    ds_up = ds.where(depth_mask_up, drop=True)
    ds_low = ds.where(depth_mask_low, drop=True)

    # 4. Compute average values in the dimension 'depth':
    mean_up = ds_up.mean(dim='depth')
    mean_low = ds_low.mean(dim='depth')

    # 5. Rename the variable names to be able to differentiate them:
    if 'Biogeochemical' in env_netcdf:
        mean_up = mean_up.rename({'chl': 'chl_up', 'nppv': 'npp_up','o2': 'o2_up','no3': 'no3_up','po4': 'po4_up','nh4': 'nh4_up','ph': 'ph_up'})
        mean_low = mean_low.rename({'chl': 'chl_low', 'nppv': 'npp_low','o2': 'o2_low','no3': 'no3_low','po4': 'po4_low','nh4': 'nh4_low','ph': 'ph_low'})
        
    elif 'Physical' in env_netcdf:
        mean_up = mean_up.rename({'so': 'so_up','thetao': 'thetao_up'})
        mean_low = mean_low.rename({'so':'so_low', 'thetao': 'thetao_low'})
        
    else: 
        print(f"{env_netcdf} netCDF file names are not included in the loop. Check the code to adapt it!")
        exit()

    # 6. Read the region of interest shapefile and check if the coordinate system of the environmental variables and the shapefile is the same:
    roi = gpd.read_file(roi_file)
    find_env_epsg = re.search(r'AUTHORITY\["EPSG","(\d+)"\]', str(mean_low.rio.crs)) # Check coordinate system of env. data
    if find_env_epsg.group(1) == roi.crs.to_epsg():
        print("The EPSG codes of the environmental variables and the region of interest are the same!")
    else: 
        print(f"The EPSG codes of the inputs are different! Environmental data projection is EPSG:{find_env_epsg.group(1)} and Region of Interest projection is EPSG:{roi.crs.to_epsg()}")
        print(f"Converting Region of Interest file projection to EPSG:{find_env_epsg.group(1)}")
        roi = roi.to_crs(epsg=find_env_epsg.group(1))

    # 7. Extract the geometry of the region of interest and clip the environmental data to the region of interest:
    roi_mask = [mapping(roi.geometry[0])]   # Extract roi geometry
    mean_up_clip = mean_up.rio.clip(roi_mask, roi.crs, drop=True)
    mean_low_clip = mean_low.rio.clip(roi_mask, roi.crs, drop=True)

    # 8. Compute the mean in each depth range and within the region of interest:
    avg_up = mean_up_clip.mean(dim=["latitude", "longitude"])
    avg_low = mean_low_clip.mean(dim=["latitude", "longitude"])

    # 9. Convert netCDF into a dataframe:
    env_dataframe_up = avg_up.to_dataframe().reset_index()
    env_dataframe_low = avg_low.to_dataframe().reset_index()

    # 10. Filter the dataframes to drop the 'spatial_ref' column that can cause problems:
    if 'spatial_ref' in env_dataframe_up.columns:
        env_dataframe_up = env_dataframe_up.drop(columns=['spatial_ref'])
    if 'spatial_ref' in env_dataframe_low.columns:
        env_dataframe_low = env_dataframe_low.drop(columns=['spatial_ref'])

    # 11. Merge the dataframes aligning the biomass estimates and environmental variable dates:
    dataset_cleared = pd.merge(dataset_cleared, env_dataframe_up, on='time', how='left')
    dataset_cleared = pd.merge(dataset_cleared, env_dataframe_low, on='time', how='left')

# 12. Create a list of the duplicate columns to merge:
cols_to_combine = ['chl_up', 'npp_up', 'o2_up', 'no3_up', 'po4_up', 'nh4_up', 'ph_up',
                   'chl_low', 'npp_low', 'o2_low', 'no3_low', 'po4_low', 'nh4_low', 'ph_low',
                   'so_up', 'thetao_up', 'so_low', 'thetao_low']

# 13. For each column, merge the versions "_x" & "_y" using combine_first():
for col in cols_to_combine:
    dataset_cleared[col] = dataset_cleared[f'{col}_x'].combine_first(dataset_cleared[f'{col}_y'])

# 14. Delete the duplicate columns (_x, _y) as the data was already merged:
dataset_cleared = dataset_cleared.drop(columns=[f'{col}_x' for col in cols_to_combine] + [f'{col}_y' for col in cols_to_combine])

# 15. Create a new column with the values of catch/Bmsy (harmonized by month) that matches 'time'
    # Read SPiCT input csv again to avoid previous modifications:
captures_data = pd.read_csv(spict_data)
    # Ensure 'timeC' is a floating type data:
captures_data['timeC'] = captures_data['timeC'].astype(float)
    # Drop NoData:
captures_data = captures_data.dropna(subset=['timeC'])
    # Create a Dataframe to store monthly means in each quarter:
monthly_captures = []

    # Loop in each quarter and compute the mean:
for index, row in captures_data.iterrows():
    year = int(row['timeC'])  
    quarter = int((row['timeC'] - year) * 100 / 25) + 1  
    
    # Calcular los meses correspondientes
    if quarter == 1:
        months = [f"{year}-01", f"{year}-02", f"{year}-03"]
    elif quarter == 2:
        months = [f"{year}-04", f"{year}-05", f"{year}-06"]
    elif quarter == 3:
        months = [f"{year}-07", f"{year}-08", f"{year}-09"]
    elif quarter == 4:
        months = [f"{year}-10", f"{year}-11", f"{year}-12"]
    
    # Asign the mean of the captures to the corresponding months:
    monthly_captures.extend([(month, row['obsC'] / 3) for month in months])

    # Convert the list to a Dataframe:
monthly_captures_df = pd.DataFrame(monthly_captures, columns=['time', 'obsC'])

    # Convert to 'time' datatype:
monthly_captures_df['time'] = pd.to_datetime(monthly_captures_df['time'])

    # Compute catch/Bmsy:
monthly_captures_df['rel_C'] = monthly_captures_df['obsC']/bmsy

    # Merge 'dataset_cleared' with 'monthly_captures_df' to add captures into the training dataset:
dataset_cleared = pd.merge(dataset_cleared, monthly_captures_df, on='time', how='left')

# 16. Drop any timestep with Nan:
dataset_cleared = dataset_cleared.dropna(how='any')

# 17. Save the dataset that will be used to train the model:
os.makedirs('Fisheries_modelling')
dataset_cleared.to_csv(os.path.join('Fisheries_modelling', 'training_dataset.csv'))

print("TRAINING DATASET PREPARED!")

## PREDICTOR SELECTION

In the following chunks we will execute the necessary methodologies to perform the selection of predictor variables.

In [54]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn

In [46]:
# 1. Load the csv with training data:
training_data = pd.read_csv(os.path.join('Fisheries_modelling', 'training_dataset.csv'))

# 2. Clear the csv if 'Unnamed' columns are in the training dataset:
training_data = training_data.loc[:, ~training_data.columns.str.contains('^Unnamed')]

# 3. Convert 'time' column into datetime type:
training_data['time'] = pd.to_datetime(training_data['time'])

# 4. Create a list with the first year of data and steps of 5 years (they will be used in the plots):
start_year = training_data['time'].dt.year.min()
end_year = training_data['time'].dt.year.max()
years = np.arange(start_year, end_year + 1, 5)

# 5. Predictor list:
predictors_names = cols_to_combine + ['previous_relative_B']  # Adjust your predictors

# 6. Create a loop to plot relative Biomass Index-Predictor Variable pairs:
for predictor in predictors_names:
    fig, ax1 = plt.subplots(figsize=(12, 6))
    
    # First axis for relative biomass:
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Relative Biomass', color='blue')
    ax1.plot(training_data['time'], training_data['relative_B'], label='Relative Biomass', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')  # Axis colour

    # Configure the ticks of the axis
    ax1.set_xticks([pd.Timestamp(f'{year}-01-01') for year in years])  # Use the list of years
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))  # Just show the year
    
    # Establish the limits for the x axis ticks:
    ax1.set_xlim([training_data['time'].min() - pd.DateOffset(years=1), training_data['time'].max()])

    
    # Rotate the xaxis ticks
    plt.xticks(rotation=45)

    # Second axis for the predictor
    ax2 = ax1.twinx()  # Create a second y axis that shares x axis
    ax2.set_ylabel(predictor, color='green')
    ax2.plot(training_data['time'], training_data[predictor], label=predictor, color='green')
    ax2.tick_params(axis='y', labelcolor='green')  # y second axis colour
    
    # Plot title
    plt.title(f'Relative Biomass and {predictor} over Time')

    # Ensure axis disposition is correct
    fig.tight_layout()
    
    # Show plot:
    plt.show()

In [ ]:
seaborn.pairplot(training_data)